In [1]:
import csv
import json
import time
import random
import pandas as pd
import multiprocessing

from tqdm import tqdm, trange
from crawler import WebDriver
from json.decoder import JSONDecodeError
from selenium.webdriver.chrome.options import Options

In [2]:
def grouping(lis, group_num):
    return [lis[i:i+group_num] for i in range(0, len(lis), group_num)]


def crawler_data(res_info):
    driver = WebDriver()
    driver.get_url(web_source)
    driver.implicitly_wait(10)
    name, address = res_info
    result = driver.search_restaurant(name, address)

    if result == (None, None):
        ratings = None
    else:
        driver.implicitly_wait(15)
        try:
            _, ratings = driver.get_reviews()
            if ratings:
                for t in ratings.keys():
                    ratings[t] = sum(ratings[t]) / len(ratings[t])

                ratings = dict(ratings)
        except:
            ratings = None
    driver.quit()
    print((name, address), result, ratings)
    return ((name, address), result, ratings)


def save_json(data, path):
    with open(path, "w") as fp:
        json.dump(data, fp)
    return

In [3]:
file_path = "ichefdata/ichefdata/ichef 資料.csv"
restaurant_num = 1 # "all"
processes_num = 1
group_num = 1
save_file = "restaurant.json"
web_source = 'https://www.google.com/search?q=mcdonald&sca_esv=556221820&tbm=lcl&ei=iSjXZMX7HMuQhwPnqJbgAg&oq=mcd&gs_lp=Eg1nd3Mtd2l6LWxvY2FsIgNtY2QqAggAMg0QABiKBRixAxiDARhDMgcQABiKBRhDMgcQABiKBRhDMgcQABiKBRhDMgcQABiKBRhDMgcQABiKBRhDMgUQABiABDIFEAAYgAQyBRAAGIAEMggQABiABBixA0jZQVAAWPg5cA94AJABAJgBKaABtgKqAQE5uAEDyAEA-AEBqAIAwgILEAAYgAQYsQMYgwHCAgsQABiKBRixAxiDAcICChAAGIoFGLEDGEOIBgE&sclient=gws-wiz-local#rlfi=hd:;si:;mv:[[25.0579939,121.5714136],[24.995991099999998,121.51168539999999]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!2m1!1e3!3sIAE,lf:1,lf_ui:4'

In [4]:
restaurant_df = pd.read_csv(file_path)
restaurant_df = restaurant_df[['店名', '店址']]
restaurant_df.rename(columns = {'店名':'name', '店址':'address'}, inplace=True)
restaurant_df = restaurant_df.drop_duplicates().reset_index(drop=True)
restaurant_df.head(10)

,name,address
0,% shabu,桃園市桃園區中埔ㄧ街331號
1,(停用）喜樂咖啡,台中市西區美村路一段687號2樓
2,055龍哥斯特,花蓮縣吉安鄉中央路三段458號
3,0870 Kitchen & Bar,台北市中山區中山北路二段137巷45號
4,1+咖啡館,高雄市苓雅區永定街91號
5,1001手作廚房,台中市西屯區成都路212號
6,101小飯廳,台北市南港區新民街101
7,108抹茶茶廊 Sogo復興店,台北市大同區華陰街91號5F
8,108抹茶茶廊 信義誠品店,台北市信義區松高路11號B2
9,11HomeKitchen,台南市中西區民族路三段128號


In [5]:
print(len(restaurant_df['name'].unique()))
print(len(restaurant_df['address'].unique()))

3778
3701


In [6]:
unique_restaurant_df = restaurant_df.groupby('address')['name'].apply(list).reset_index(name='name_list')
unique_restaurant_df

,address,name_list
0,0,[NDBC-VN]
1,100台灣台北市中正區濟南路二段48巷3-2號,[禮面作]
2,103台灣台北市大同區延平北路一段69巷5號2F,[Retro印刷]
3,104台北市中山區一江街25號1樓,[parc.c.studio]
4,104台北市中山區雙城街13巷18號2樓 (平信 18:00開店),[Ansleep]
...,...,...
3695,高雄市鼓山區青海路277號,[BoBo Rose]
3696,高雄市鼓山區鼓元街57號一樓,[My Cofi 哈瑪星]
3697,高雄市鼓山區鼓南街30號1樓,[木葉粗食]
3698,高雄市鼓山區鼓山一路53巷107號,[光悅茶屋]


In [7]:
unique_restaurant_df[unique_restaurant_df["name_list"].apply(lambda x: len(x)) > 1]

,address,name_list
72,南投縣草屯鎮墩煌路三段168號,"[督賀咖啡, 督賀蔬苑]"
180,台中市北屯區昌平路一段105-12號,"[威霸鍋物, 米可烘焙坊]"
194,台中市北屯區熱河路二段128號,"[憲賣咖啡, 憲賣咖啡(外帶)]"
270,台中市后里區月眉北路486號,"[星月大地 咖啡廳, 星月大地 售票處, 星月大地 宴會館]"
293,台中市太平區中山路四段170號,"[漢家園 中山店 第一結帳口, 漢家園 中山店 第二結帳口]"
...,...,...
3524,高雄市左營區富國路420,"[北方拉麵三, 北方拉麵之家]"
3525,高雄市左營區富國路420號,"[北方拉麵之家, 北方拉麵四]"
3545,高雄市左營區重信路557號4樓,"[墨凡有限公司, 墨凡有限公司窗口三]"
3578,高雄市楠梓區壽民路51號,"[小本愛玉 大樹店, 小本愛玉 楠梓店]"


In [8]:
restaurant_list = list(zip(restaurant_df['name'], restaurant_df['address']))
restaurant_list = restaurant_list if restaurant_num == "all" else restaurant_list[:restaurant_num]

In [9]:
restaurant_group_list = grouping(restaurant_list, group_num)

In [10]:
start_time = time.time()
result_list = []

for group in restaurant_group_list:
    pool = multiprocessing.Pool(processes=processes_num)
    result = pool.map(crawler_data, group)
    result_list.extend(result)

print(time.time() - start_time)


Scrolling: 100%|████████████████████████████████████████████████████████████████████| 899/899 [01:58<00:00,  5.56it/s]
                                                                                                                      
Get review: 100%|██████████████████████████████████████████████████████████████████▊| 897/899 [00:21<00:00, 46.62it/s]
                                                                                                                      

('% shabu', '桃園市桃園區中埔ㄧ街331號') ('% Percent Shabu精品鍋物', '中埔ㄧ街331號1樓') {'2 週前': 5.0, '1 個月前': 5.0, '2 個月前': 5.0, '4 個月前': 5.0, '6 個月前': 1.0, '7 個月前': 2.0, '8 個月前': 4.75, '9 個月前': 5.0, '10 個月前': 2.0, '11 個月前': 5.0, '1 年前': 4.222222222222222, '2 年前': 4.309523809523809, '3 年前': 4.4875, '4 年前': 4.920560747663552, '5 年前': 4.971337579617835}
152.3782444000244


In [11]:
result_dict = {}
for (name, address), result, ratings in result_list:
    result_dict[name] = ((name, address), result, ratings)
    save_json(result_dict, save_file)